In [3]:
#!pip install btrack

In [2]:
import os
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack

In [3]:
df_dir = r'Z:\COOK_LAB\CELL_TRACKING\example_results\df'
df_file = 'cellPose_regionprops.pkl'

save_dir = df_dir

In [4]:
modelPath = r'Z:\COOK_LAB\CELL_TRACKING\code\cell_config.json'

## Read in the data frame objects data

In [5]:
data_df = pd.read_pickle(os.path.join(df_dir,df_file))

In [6]:
data_df.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,bbox-3,image,file,t,size_x,size_y
0,3,670,21.959701,863.031343,0.859500,38.916625,22.064443,7,847,38,880,"[[False, False, False, False, False, False, Fa...",01_ch0_000.png,0,1946,1946
1,4,646,45.170279,1153.159443,-0.660946,33.668349,24.462243,31,1140,61,1168,"[[False, False, False, False, False, False, Fa...",01_ch0_000.png,0,1946,1946
2,5,770,50.325974,394.742857,-0.086673,35.855595,27.370902,33,382,69,409,"[[False, False, False, False, False, False, Fa...",01_ch0_000.png,0,1946,1946
3,6,696,58.857759,1110.521552,1.420108,33.783068,26.455876,46,1095,73,1128,"[[False, False, False, False, False, False, Fa...",01_ch0_000.png,0,1946,1946
4,7,809,74.687268,466.436341,0.680750,34.879034,29.537244,59,451,92,483,"[[False, False, False, False, False, False, Fa...",01_ch0_000.png,0,1946,1946


In [7]:
# create a structure suitable for tracking

objects_gen = data_df.loc[:,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
objects_gen['z']=0
objects_gen['label']=5
objects_gen['prob']=0
objects_gen['dummy']=False
objects_gen['states']=0

objects_gen.head()

,ID,area,x,y,major_axis_length,minor_axis_length,t,z,label,prob,dummy,states
0,3,670,863.031343,21.959701,38.916625,22.064443,0,0,5,0,False,0
1,4,646,1153.159443,45.170279,33.668349,24.462243,0,0,5,0,False,0
2,5,770,394.742857,50.325974,35.855595,27.370902,0,0,5,0,False,0
3,6,696,1110.521552,58.857759,33.783068,26.455876,0,0,5,0,False,0
4,7,809,466.436341,74.687268,34.879034,29.537244,0,0,5,0,False,0


## Tracking proper

In [8]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(modelPath)

    # append the objects to be tracked
    tracker.append(objects_gen)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # get the tracks as a python list
    tracks = tracker.tracks

    # optional: get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)
    # pickle Napari data
    with open(os.path.join(df_dir,'track.pkl'),'wb') as f:
        pickle.dump([data,properties,graph],f)

[INFO][2021/09/16 02:48:09 PM] Loaded btrack: C:\ProgramData\Anaconda3\envs\cellwork\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/09/16 02:48:09 PM] btrack (v0.4.2) library imported
[INFO][2021/09/16 02:48:09 PM] Setting max XYZ search radius to: 100
[INFO][2021/09/16 02:48:09 PM] Starting BayesianTracker session
[INFO][2021/09/16 02:48:09 PM] Loading configuration file: Z:\COOK_LAB\CELL_TRACKING\code\cell_config.json
[INFO][2021/09/16 02:48:09 PM] Loading motion model: b'cell_motion'
[INFO][2021/09/16 02:48:09 PM] Objects are of type: <class 'pandas.core.frame.DataFrame'>
[INFO][2021/09/16 02:48:09 PM] Set volume to ((0, 1946), (0, 1946), (-100000.0, 100000.0))
[INFO][2021/09/16 02:48:09 PM] Starting tracking... 
[INFO][2021/09/16 02:48:09 PM] Tracking objects in frames 0 to 25 (of 25)...
[INFO][2021/09/16 02:48:09 PM]  - Timing (Bayesian updates: 12.00ms, Linking: 1.00ms)
[INFO][2021/09/16 02:48:09 PM]  - Probabilities (Link: 1.00000, Lost: 0.98897)
[INFO][2021/09/16 02:4

## Merging objects and tracking information

In [9]:
trackDataAll = pd.DataFrame(data,columns=['track_id','t','x','y'])
trackDataAll['parent'] = properties['parent']
trackDataAll['generation'] = properties['generation']
trackDataAll['root'] = properties['root']

In [10]:
len(trackDataAll)

3774

In [11]:
trackDataAll

,track_id,t,x,y,parent,generation,root
0,1.0,0.0,1002.566667,596.288889,1,0,1
1,1.0,1.0,1001.945364,591.389073,1,0,1
2,1.0,2.0,1006.442464,587.599676,1,0,1
3,1.0,3.0,1008.517298,588.004942,1,0,1
4,1.0,4.0,1009.160279,592.202091,1,0,1
...,...,...,...,...,...,...,...
3769,241.0,24.0,724.958054,667.966443,165,1,165
3770,242.0,24.0,1620.496979,1059.610272,242,0,242
3771,243.0,24.0,24.318367,1148.838776,243,0,243
3772,245.0,24.0,127.219931,1093.542955,111,1,111


In [12]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [17]:
print(f'Number of all objects: {len(allData)}')

Number of all objects: 3740


In [13]:
allData.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,file,t,size_x,size_y,track_id,x,y,parent,generation,root
0,3,670,21.959701,863.031343,0.859500,38.916625,22.064443,7,847,38,...,01_ch0_000.png,0,1946,1946,57.0,21.959701,863.031343,57.0,0.0,57.0
1,4,646,45.170279,1153.159443,-0.660946,33.668349,24.462243,31,1140,61,...,01_ch0_000.png,0,1946,1946,67.0,45.170279,1153.159443,67.0,0.0,67.0
2,5,770,50.325974,394.742857,-0.086673,35.855595,27.370902,33,382,69,...,01_ch0_000.png,0,1946,1946,97.0,50.325974,394.742857,97.0,0.0,97.0
3,6,696,58.857759,1110.521552,1.420108,33.783068,26.455876,46,1095,73,...,01_ch0_000.png,0,1946,1946,98.0,58.857759,1110.521552,98.0,0.0,98.0
4,7,809,74.687268,466.436341,0.680750,34.879034,29.537244,59,451,92,...,01_ch0_000.png,0,1946,1946,99.0,74.687268,466.436341,99.0,0.0,99.0


In [14]:
allData.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'orientation',
       'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2',
       'bbox-3', 'image', 'file', 't', 'size_x', 'size_y', 'track_id', 'x',
       'y', 'parent', 'generation', 'root'],
      dtype='object')

## Visualize (optional)

In [19]:
im_dir = r'Z:\COOK_LAB\CELL_TRACKING\example_results\results_01_full'
file_list = [x for x in os.listdir(im_dir) if 'png' in x]

In [20]:
# example runs for the first 'num_im' images
labels = []

num_im = 25
for myFile in file_list[:num_im]:
    # set pathway to an image
    myFilePath = os.path.join(im_dir,myFile)
    # read image
    labels_2D = imread(myFilePath)
    # put the image into the list
    labels.append(labels_2D)
    
labels = np.array(labels)
labels.shape

(25, 1946, 1946)

In [21]:
viewer = napari.Viewer()
viewer.add_labels(labels)
viewer.add_tracks(data, properties=properties, graph=graph)

C:\ProgramData\Anaconda3\envs\cellwork\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Tracks layer 'data' at 0x1f12fa58d00>

In [17]:
%%time
myLabels =[]

for j in range(startFrame,stopFrame):
    
    # read a mask
    labels = imread(os.path.join(myDirLabels,myFile.replace('.tif',f'_{str(j).zfill(3)}_label.tif')))
    
    myLabels.append(labels)

NameError: name 'startFrame' is not defined